# EDA Analysis and model experimentation #

In [1]:
# Importing libraries
import os
import pandas as pd
import numpy as np

from src import config
import matplotlib.pyplot as plt
import seaborn as sns

import openai
import pandas as pd
import boto3
import langchain

# Importing ENV Variables

In [2]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
OPEN_AI_API_KEY = os.getenv('OPEN_AI_API_KEY') # API key for OpenAI
AWS_S3= os.getenv('AWS_S3')
AWS_S3_SECRET = os.getenv('AWS_S3_SECRET')

## Instanciating a connection to S3 with credentials

In [4]:

s3 = boto3.client('s3', 
                  aws_access_key_id=AWS_S3, 
                  aws_secret_access_key=AWS_S3_SECRET
                  )


## Getting the files filter by prefix

In [5]:

from src import config
from pathlib import Path
prefix = 'queplan_insurance/'
bucket_name = 'anyoneai-datasets'   
maxkeys = 9999999
s3 = boto3.client('s3')
list=s3.list_objects(Bucket=bucket_name, Prefix=prefix, MaxKeys = maxkeys)['Contents']
list_keys = []
list_filepath = []
dataset_path = str(Path(config.DATASET_ROOT_PATH) / "raw_pdfs")

for s3_key in list:
    s3_object = s3_key['Key']
    
    path, filename = os.path.split(s3_object)
    
    filepath = os.path.join(dataset_path, filename)
    list_filepath.append(filepath) 
    list_keys.append(s3_object)

In [6]:
list_filepath

['/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL120190177.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320130223.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320180100.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
 '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320200071.pdf',
 '/home/andy/In

In [7]:
list_keys

['queplan_insurance/',
 'queplan_insurance/POL120190177.pdf',
 'queplan_insurance/POL320130223.pdf',
 'queplan_insurance/POL320150503.pdf',
 'queplan_insurance/POL320180100.pdf',
 'queplan_insurance/POL320190074.pdf',
 'queplan_insurance/POL320200071.pdf',
 'queplan_insurance/POL320200214.pdf',
 'queplan_insurance/POL320210063.pdf',
 'queplan_insurance/POL320210210.pdf']

## Downloading the specific files by prefix 'queplan_insurance'

In [8]:
s3 = boto3.resource('s3')

for key,path in zip(list_keys[1:],list_filepath[1:]):
    if os.path.exists(path):
        print(f"File already downloaded: {path}")
    else:
        s3.meta.client.download_file(bucket_name, key,path)

File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL120190177.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320130223.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320180100.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf
File already downloaded: /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320200071.pdf
File already dow

In [9]:
# [TO DO] - Add code to download the data from S3 bucket as a function in data utils
# def download_data_from_s3():

In [10]:
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader
pdf_list = os.listdir(str(Path(config.DATASET_ROOT_PATH) /  "raw_pdfs"))

list_docs = []
pdf_list


['POL320150503.pdf',
 'POL320190074.pdf',
 'POL320210063.pdf',
 'POL320180100.pdf',
 'POL320210210.pdf',
 'POL320130223.pdf',
 'POL320200214.pdf',
 'POL320200071.pdf',
 'POL120190177.pdf']

In [11]:
config.DATASET_ROOT_PATH

'/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset'

In [12]:
for pdf in pdf_list[1:]:
    loader_pymu = PyMuPDFLoader(f"{config.DATASET_ROOT_PATH}/raw_pdfs/{pdf}")  # Load the document
    pages_pymu = loader_pymu.load()
    list_docs.append(pages_pymu)

In [13]:
list_docs[5][0]

Document(page_content='SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO\nIncorporada al Depósito de Pólizas bajo el código POL320200214\n \nARTICULO 1°: REGLAS APLICABLES AL CONTRATO\n \n \nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o dependiente.\n \n \nLa presente póliza se otorga en base a las declaraciones, informaciones y antecedentes proporcionados por\nel asegurado titular a solicitud del Asegurador, y en base a la información que ha entregado el Asegurador al\nasegurado titular respecto a las condiciones, términos y modalidades del seguro, todos los cuales forman\nparte integrante de la presente póliza.\n \n \nARTÍCULO 2º: COBERTURA\n \n \nEl Asegurador reembolsará los gastos médicos razonables, ac

In [14]:
pdf_list

['POL320150503.pdf',
 'POL320190074.pdf',
 'POL320210063.pdf',
 'POL320180100.pdf',
 'POL320210210.pdf',
 'POL320130223.pdf',
 'POL320200214.pdf',
 'POL320200071.pdf',
 'POL120190177.pdf']

In [15]:
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_pdfs/")
pages_dl = loader_dl.load()

In [16]:
pages_dl[5].metadata["source"]

'dataset/raw_pdfs/POL320130223.pdf'

In [17]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_pdfs/")
pages_dl = loader_dl.load()

MODEL = "gpt-3.5-turbo"

encoder = tiktoken.encoding_for_model(MODEL)


tokens_dict = {}
for page in pages_dl:
    num_tokens = len(encoder.encode(page.page_content))
    tokens_dict[page.metadata["source"]] = num_tokens

In [18]:
tokens_dict

{'dataset/raw_pdfs/POL320150503.pdf': 16949,
 'dataset/raw_pdfs/POL320190074.pdf': 28058,
 'dataset/raw_pdfs/POL320210063.pdf': 4192,
 'dataset/raw_pdfs/POL320180100.pdf': 15782,
 'dataset/raw_pdfs/POL320210210.pdf': 6838,
 'dataset/raw_pdfs/POL320130223.pdf': 13783,
 'dataset/raw_pdfs/POL320200214.pdf': 27308,
 'dataset/raw_pdfs/POL320200071.pdf': 23280,
 'dataset/raw_pdfs/POL120190177.pdf': 16037}

In [19]:
pages_dl[1].page_content

'SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO\n\nIncorporada al Depósito de Pólizas bajo el código POL320190074\n\nARTICULO 1°: REGLAS APLICABLES AL CONTRATO\n\nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o beneficiario.\n\nARTÍCULO 2º: COBERTURA\n\nLa compañía aseguradora reembolsará los gastos médicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un Evento cubierto por esta póliza, en los términos y condiciones establecidas en ésta, siempre que haya transcurrido el periodo de carencia establecido en las Condiciones Particulares, que la póliza se encuentre vigente y que no haya transcurrido el plazo definido en las Condiciones Particulares para la cobertura del Evento.

In [20]:
import importlib
import src.text_preprocessing as tp

[nltk_data] Downloading package stopwords to /home/andy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/andy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


    corpus: List[str],
    html_stripping: Optional[bool] = True,
    accented_char_removal: Optional[bool] = True,
    text_lower_case: Optional[bool] = True,
    text_stemming: Optional[bool] = False,
    text_lemmatization: Optional[bool] = False,
    special_char_removal: Optional[bool] = True,
    remove_digits: Optional[bool] = True,
    stopword_removal: Optional[bool] = True,
    stopwords: Optional[List[str]] = stopword_list,

In [21]:
text = pages_dl[1].page_content

In [22]:
text_wo_char = tp.normalize_corpus([text], special_char_removal=True, stopword_removal=False, remove_digits=False)
text_wo_char , len(text_wo_char[0])

(['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo pol320190074\n\narticulo 1 reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii del libro ii del codigo de comercio sin embargo se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario\n\narticulo 2o cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables acostumbrados y efectivamente incurridos por el asegurado asociados a un evento cubierto por esta poliza en los terminos y condiciones establecidas en esta siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura del evento\n\nlos reem

In [23]:
text_w_char = tp.normalize_corpus([text], special_char_removal=False, stopword_removal=False,remove_digits=False)
text_w_char , len(text_w_char[0])

(['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo pol320190074\n\narticulo 1°: reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario.\n\narticulo 2o: cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un evento cubierto por esta poliza, en los terminos y condiciones establecidas en esta, siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares, que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura del event

In [24]:
import re
text_w_char = tp.normalize_corpus([text], special_char_removal=False, stopword_removal=False,remove_digits=False)
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{1}\s*(\s*[^\n]*)((?=[\s*]*))"
articles = re.findall(pattern, text_w_char[0])

In [25]:
print(articles)

[('\narticulo 1', 'reglas aplicables al contrato', ''), ('\narticulo 2', 'cobertura', ''), ('\narticulo 3', 'definiciones.', ''), ('\narticulo 4', 'exclusiones.', ''), ('\narticulo 5', 'carencia', ''), ('\narticulo 6', 'obligaciones del asegurado', ''), ('\narticulo 7', 'declaraciones del asegurado', ''), ('\narticulo 8', 'primas y efectos del no pago de las primas', ''), ('\narticulo 9', 'beneficiarios', ''), ('\narticulo 10', 'denuncia de siniestros', ''), ('\narticulo 11', 'vigencia y terminacion', ''), ('\narticulo 12', 'comunicacion entre las partes', ''), ('\narticulo 13', 'solucion de controversias', ''), ('\narticulo 14', 'clausulas adicionales', ''), ('\narticulo 15', 'domicilio', ''), ('\narticulo n° 1', 'cobertura.', ''), ('\narticulo n° 2', 'exclusiones.', ''), ('\narticulo n° 3', 'carencia', ''), ('\narticulo no 4', 'beneficiarios', ''), ('\narticulo n° 5', 'denuncia de siniestro', ''), ('\narticulo 6', 'clausulas aplicables', ''), ('\narticulo 1', 'reglas aplicables al co

In [26]:
pattern_policies = r"([a-z]{3}\d+)"
match_dict = {}
for doc in text_w_char:
    matches = re.split(pattern_policies, text_w_char[0])

In [27]:
matches

['seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo ',
 'pol320190074',
 '\n\narticulo 1°: reglas aplicables al contrato\n\nse aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o beneficiario.\n\narticulo 2o: cobertura\n\nla compania aseguradora reembolsara los gastos medicos razonables, acostumbrados y efectivamente incurridos por el asegurado, asociados a un evento cubierto por esta poliza, en los terminos y condiciones establecidas en esta, siempre que haya transcurrido el periodo de carencia establecido en las condiciones particulares, que la poliza se encuentre vigente y que no haya transcurrido el plazo definido en las condiciones particulares para la cobertura 

In [28]:
import PyPDF2

policies = {}
policies_list = []
for pdf in pdf_list:
    pattern = r"([A-za-z]{3}\d{3,15})"
    filepath = str(Path(config.DATASET_ROOT_PATH) /  f"raw_pdfs/{pdf}")
    with open(filepath, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pattern_found = False
        start_page = None
        

        policies_metadata = {"policy_number": None, "start_page": None, "end_page": None, "filepath": None, "num_pages": None}
        num_pages = len(pdf_reader.pages)
        policies_num = 0
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            pattern_found = re.findall(pattern, page_text)
            
            
            if pattern_found:
                if policies_num == 0:
                    policies_num += 1
                    policies_metadata = {"policy_number": pattern_found[0], "start_page": page_num, "end_page": None, "filepath": filepath, "num_pages": None}
                    policies_list.append(policies_metadata)
                elif policies_num > 0:
                    policies_num += 1
                    end_page = page_num
                    policies_list[-1]["end_page"] = end_page-1
                    start_page = policies_list[-1]["start_page"]
                    num_pages = end_page - start_page
                    policies_list[-1]["num_pages"] = num_pages
                    policies_metadata = {"policy_number": pattern_found[0], "start_page": page_num, "end_page": None, "filepath": filepath, "num_pages": None}
                    policies_list.append(policies_metadata)
    policies_list[-1]["num_pages"] = num_pages
    policies_list[-1]["end_page"] = page_num
            

In [29]:
policies_list

[{'policy_number': 'POL320150503',
  'start_page': 0,
  'end_page': 25,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf',
  'num_pages': 26},
 {'policy_number': 'POL320190074',
  'start_page': 0,
  'end_page': 9,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 10},
 {'policy_number': 'CAD220130244',
  'start_page': 10,
  'end_page': 14,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 5},
 {'policy_number': 'POL320130223',
  'start_page': 15,
  'end_page': 61,
  'filepath': '/home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf',
  'num_pages': 47},
 {'polic

In [30]:
for doc in policies_list:
    pn, sp, ep, fp, np = doc.values()
    with open(fp, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        output_pdf = PyPDF2.PdfWriter()
        output_file = f'{config.DATASET_ROOT_PATH}/raw_chunks/{pn}.pdf'
        if not os.path.exists(output_file):
            for i in range(sp, ep+1):
                output_pdf.add_page(pdf_reader.pages[i])
            with open(f'{config.DATASET_ROOT_PATH}/raw_chunks/{pn}.pdf', 'wb') as output_file:
                    output_pdf.write(output_file)
        else:
            print(f"""File {pn} already exists.
                  check manually for inconsistencies in file {fp} with the following original raw doc""")

File POL320150503 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320150503.pdf with the following original raw doc
File POL320190074 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf with the following original raw doc
File CAD220130244 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_experiments/dataset/raw_pdfs/POL320190074.pdf with the following original raw doc
File POL320130223 already exists.
                  check manually for inconsistencies in file /home/andy/Insync/anvasquezre@unal.edu.co/Google Drive/ML_AI/AnyoneAI/06_Final_Project/model_ex

In [31]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
loader_dl = DirectoryLoader("dataset/raw_chunks/")
pages_dl = loader_dl.load()

EMBEDDINGS_MODEL = "text-embedding-ada-002"
MODEL = "gpt-3.5-turbo"
MODEL_16K = "gpt-3.5-turbo-16k"

encoder = tiktoken.encoding_for_model(EMBEDDINGS_MODEL)

In [32]:
tokens_dict = {}
for doc in pages_dl:
    content = doc.page_content
    num_tokens = len(encoder.encode(content))
    text_w_char = tp.normalize_corpus([content], special_char_removal=False, stopword_removal=False,remove_digits=False)
    pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*(\s*[^\n]*)((?=[\s*]*))"
    articles = len(re.findall(pattern, text_w_char[0]))
    tokens_dict[doc.metadata["source"]] = {"num_tokens": num_tokens , "num_articles": articles}
tokens_dict

{'dataset/raw_chunks/CAD220130244.pdf': {'num_tokens': 1912,
  'num_articles': 6},
 'dataset/raw_chunks/CAD320190121.pdf': {'num_tokens': 3656,
  'num_articles': 6},
 'dataset/raw_chunks/POL320160108.pdf': {'num_tokens': 7987,
  'num_articles': 20},
 'dataset/raw_chunks/POL320150503.pdf': {'num_tokens': 16949,
  'num_articles': 23},
 'dataset/raw_chunks/POL320190074.pdf': {'num_tokens': 6806,
  'num_articles': 15},
 'dataset/raw_chunks/CAD220130227.pdf': {'num_tokens': 1901,
  'num_articles': 6},
 'dataset/raw_chunks/POL320210063.pdf': {'num_tokens': 4192,
  'num_articles': 15},
 'dataset/raw_chunks/POL320180100.pdf': {'num_tokens': 15782,
  'num_articles': 23},
 'dataset/raw_chunks/POL320210210.pdf': {'num_tokens': 6838,
  'num_articles': 15},
 'dataset/raw_chunks/POL320130223.pdf': {'num_tokens': 13783,
  'num_articles': 19},
 'dataset/raw_chunks/POL320200214.pdf': {'num_tokens': 27308,
  'num_articles': 20},
 'dataset/raw_chunks/POL320200071.pdf': {'num_tokens': 15293,
  'num_articl

In [33]:
text_dict = {}
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*(\s*[^\n]*)((?=[\s*]*))"

for doc in pages_dl:
    content = doc.page_content
    text_w_char = tp.normalize_corpus([content], special_char_removal=False, stopword_removal=False,remove_digits=False)
    num_tokens = len(encoder.encode(content))
    articles = len(re.findall(pattern, text_w_char[0]))
    title = text_w_char[0].split(r"articulo")[0]
    text_dict[doc.metadata["source"]] = {"num_tokens": num_tokens , "num_articles": articles, "text": text_w_char, "title": title}
text_dict


{'dataset/raw_chunks/CAD220130244.pdf': {'num_tokens': 1912,
  'num_articles': 6,
  'text': ['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pag

In [34]:
text = text_dict['dataset/raw_chunks/CAD220130244.pdf']['text']

In [35]:
import re
pattern = r"(\narticulo .*.*\s*\d+\d*).*:{0,1}\.*\s*"
pattern = r"(?<=\n)(articulo .*.*\s*\d+\d*).*:{0,1}\.*\s*"
re.split(pattern=pattern, string=text[0])

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\n',
 'articulo n° 1',
 'en caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efectos de

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

pattern = r"(?<=\n)articulo .*.*\s*\d+\d*.*:{0,1}\.*\s*"

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100 ,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len)

In [37]:
text[0]

'exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efect

In [38]:
r_splitter.split_text(text[0])

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efec

In [39]:
from langchain.text_splitter import CharacterTextSplitter

tiktoken_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    model_name=EMBEDDINGS_MODEL,
    chunk_size=2000,
    chunk_overlap=500,
    separator="[articulo]{0,1}[ARTICULO]{0,1}[ARTÍCULO]{0,1}[Artículo]{0,1}",
    keep_separator=True,
)

In [40]:
texts = tiktoken_splitter.split_text(text[0])
texts

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\narticulo n° 1: cobertura.\n\nen caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente clausula adicional, el grupo de personas que figuran como asegurados dependientes en la poliza principal quedara exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y segun lo establecido en las condiciones particulares de la poliza.\n\npara los efectos de esta clausula adicional se entiende por asegurado la persona designada en calidad de asegurado titular para la cobertura principal, quien debera tener a su cargo la obligacion del pago de las primas de la cobertura contratada.\n\nel valor de las primas a considerar para efec

In [41]:
token_splits = tiktoken_splitter.split_documents(pages_dl)

In [42]:
token_splits

[Document(page_content='EXONERACIÓN DE PAGO DE PRIMAS POR FALLECIMIENTO DEL ASEGURADO\n\nTITULAR\n\nIncorporada al Depósito de Pólizas bajo el código CAD220130244\n\nARTICULO N° 1: COBERTURA.\n\nEn caso de producirse el fallecimiento del asegurado titular de la cobertura principal durante la vigencia de la presente Cláusula Adicional, el grupo de personas que figuran como asegurados dependientes en la póliza principal quedará exonerado del pago de todas y cada una de las primas que por concepto de las coberturas que hubieren sido contratadas se devenguen con posterioridad a la fecha de fallecimiento del asegurado titular, por el periodo de tiempo y según lo establecido en las Condiciones Particulares de la Póliza.\n\nPara los efectos de esta Cláusula Adicional se entiende por asegurado la persona designada en calidad de Asegurado Titular para la cobertura principal, quien deberá tener a su cargo la obligación del pago de las primas de la cobertura contratada.\n\nEl valor de las primas 

In [43]:
encoder = tiktoken.encoding_for_model(MODEL)
def count_tokens(str):
    return len(encoder.encode(str))

In [44]:
doc_tokens = []
for doc in token_splits:
    tokens  = count_tokens(doc.page_content)
    doc_tokens.append(tokens)
    
doc_tokens

[605,
 592,
 628,
 536,
 614,
 589,
 608,
 604,
 597,
 623,
 597,
 460,
 608,
 640,
 613,
 592,
 640,
 613,
 576,
 598,
 584,
 590,
 610,
 621,
 627,
 604,
 575,
 589,
 594,
 282,
 630,
 645,
 601,
 598,
 577,
 577,
 607,
 613,
 603,
 603,
 603,
 562,
 585,
 558,
 592,
 583,
 625,
 638,
 619,
 618,
 582,
 610,
 625,
 582,
 601,
 608,
 608,
 623,
 624,
 603,
 626,
 597,
 622,
 605,
 601,
 607,
 611,
 205,
 613,
 577,
 612,
 576,
 546,
 582,
 625,
 606,
 639,
 622,
 591,
 593,
 569,
 572,
 558,
 605,
 591,
 632,
 524,
 606,
 579,
 576,
 577,
 573,
 551,
 548,
 554,
 603,
 323,
 626,
 633,
 612,
 616,
 618,
 624,
 606,
 607,
 602,
 569,
 566,
 545,
 601,
 569,
 617,
 620,
 599,
 653,
 637,
 616,
 584,
 605,
 595,
 608,
 635,
 626,
 609,
 615,
 608,
 611,
 602,
 610,
 605,
 586,
 314,
 611,
 576,
 612,
 576,
 546,
 574,
 621,
 605,
 635,
 631,
 587,
 591,
 571,
 568,
 591,
 605,
 618,
 605,
 613,
 609,
 582,
 585,
 602,
 592,
 618,
 607,
 575,
 577,
 567,
 610,
 603,
 614,
 597,
 622,
 627

In [45]:
splits = r_splitter.split_documents(pages_dl)

In [46]:

from langchain.vectorstores import Chroma

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [48]:
embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")

/home/andy/miniconda3/envs/chat_bot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# db = Chroma.from_documents(token_splits,persist_directory="chroma", embedding=embeddings)

In [50]:
# db.persist()

In [51]:
summary_df = pd.DataFrame().from_dict(text_dict, orient='index').reset_index(drop=False)
summary_df

,index,num_tokens,num_articles,text,title
0,dataset/raw_chunks/CAD220130244.pdf,1912,6,[exoneracion de pago de primas por fallecimien...,exoneracion de pago de primas por fallecimient...
1,dataset/raw_chunks/CAD320190121.pdf,3656,6,[clausula de invalidez permanente 80% por acci...,clausula de invalidez permanente 80% por accid...
2,dataset/raw_chunks/POL320160108.pdf,7987,20,[seguro individual de enfermedades graves\n\ni...,seguro individual de enfermedades graves\n\nin...
3,dataset/raw_chunks/POL320150503.pdf,16949,23,[poliza de seguro para prestaciones medicas de...,poliza de seguro para prestaciones medicas der...
4,dataset/raw_chunks/POL320190074.pdf,6806,15,[seguro para prestaciones medicas de alto cost...,seguro para prestaciones medicas de alto costo...
5,dataset/raw_chunks/CAD220130227.pdf,1901,6,[exoneracion de pago de primas por fallecimien...,exoneracion de pago de primas por fallecimient...
6,dataset/raw_chunks/POL320210063.pdf,4192,15,[seguro individual obligatorio de salud asocia...,seguro individual obligatorio de salud asociad...
7,dataset/raw_chunks/POL320180100.pdf,15782,23,[poliza de seguro para prestaciones medicas de...,poliza de seguro para prestaciones medicas der...
8,dataset/raw_chunks/POL320210210.pdf,6838,15,[seguro para prestaciones medicas de alto cost...,seguro para prestaciones medicas de alto costo...
9,dataset/raw_chunks/POL320130223.pdf,13783,19,[seguro colectivo complementario de salud\n\ni...,seguro colectivo complementario de salud\n\nin...


In [52]:
titles = summary_df['title'].to_list()
titles

['exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad220130244\n\n',
 'clausula de invalidez permanente 80% por accidente o enfermedad\n\nincorporada al deposito de polizas bajo el codigo cad320190121\n\nesta clausula adicional es parte integrante y accesoria de la poliza principal. en consecuencia, se regira por las condiciones generales de dicha poliza y por lo dispuesto en los siguientes ',
 'seguro individual de enfermedades graves\n\nincorporada al deposito de polizas bajo el codigo pol320160108\n\n',
 'poliza de seguro para prestaciones medicas derivadas de accidente\n\ny enfermedad\n\nincorporada al deposito de polizas bajo el codigo pol320150503\n\n',
 'seguro para prestaciones medicas de alto costo\n\nincorporada al deposito de polizas bajo el codigo pol320190074\n\n',
 'exoneracion de pago de primas por fallecimiento del asegurado\n\ntitular\n\nincorporada al deposito de polizas bajo el codigo cad22

In [53]:
metadata = summary_df[['index','num_tokens','num_articles']].rename({"index": "source"},axis=1).to_dict(orient='records')
metadata

[{'source': 'dataset/raw_chunks/CAD220130244.pdf',
  'num_tokens': 1912,
  'num_articles': 6},
 {'source': 'dataset/raw_chunks/CAD320190121.pdf',
  'num_tokens': 3656,
  'num_articles': 6},
 {'source': 'dataset/raw_chunks/POL320160108.pdf',
  'num_tokens': 7987,
  'num_articles': 20},
 {'source': 'dataset/raw_chunks/POL320150503.pdf',
  'num_tokens': 16949,
  'num_articles': 23},
 {'source': 'dataset/raw_chunks/POL320190074.pdf',
  'num_tokens': 6806,
  'num_articles': 15},
 {'source': 'dataset/raw_chunks/CAD220130227.pdf',
  'num_tokens': 1901,
  'num_articles': 6},
 {'source': 'dataset/raw_chunks/POL320210063.pdf',
  'num_tokens': 4192,
  'num_articles': 15},
 {'source': 'dataset/raw_chunks/POL320180100.pdf',
  'num_tokens': 15782,
  'num_articles': 23},
 {'source': 'dataset/raw_chunks/POL320210210.pdf',
  'num_tokens': 6838,
  'num_articles': 15},
 {'source': 'dataset/raw_chunks/POL320130223.pdf',
  'num_tokens': 13783,
  'num_articles': 19},
 {'source': 'dataset/raw_chunks/POL32020

In [54]:
# db_summarized = Chroma.from_texts(
#     texts=titles,
#     metadatas=metadata,
#     persist_directory="chroma", 
#     embedding=embeddings,
#     collection_name='policies_info',
#     )

In [55]:
db_summarized = Chroma(persist_directory="chroma",
    embedding_function=embeddings, 
    collection_name="policies_info"
    )

In [56]:
db = Chroma(
    persist_directory="chroma",
    embedding_function=embeddings, 
    collection_name="langchain"
    )

In [57]:
# query it
query = "Que cubre la poliza POL120190177 PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS"
docs = db.similarity_search(query)

# print results
[doc.page_content for doc in docs]

['blación.\n\nPERÍODO DE GRACIA: El período de tiempo de 30 (treinta) días después de la fecha de vencimiento de la prima, durante el cual el Asegurador permitirá que la póliza sea pagada.\n\nPÓLIZA: Su contrato de seguro con el Asegurador, tal como está descrito en el Artículo 1 de este documento.\n\nPRESTACIÓN PROGRESIVA: Se refiere a coberturas especificadas en el Condicionado General o en el Condicionado Particular de la póliza cuyo acceso para el asegurado está asociado al transcurso de la vigencia de la póliza, sin cambios en el valor de la prima acordada durante la vigencia del seguro y sin perjuicio de los ajustes que pueda experimentar en las renovaciones, si procede.\n\nPRUEBA CLÍNICA REGISTRADA: Cualquier prueba clínicamente controlada y éticamente aprobada que esté incluida en un registro de información nacional o internacional de pruebas clínicas (por ejemplo: www.clinicaltrials.gov, www.ISRCTN.org o www.ukctg.nihr.ac.uk).\n\nPSIQUIATRA, PSICÓLOGO Y PSICOTERAPEUTA: Profesi

In [58]:
# query it
query = "Que cubre la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS"
docs = db_summarized.similarity_search_with_relevance_scores(query, k = 1)

# print results
[doc for doc in docs]

[(Document(page_content='poliza de accidentes personales / reembolso gastos medicos\n\nincorporada al deposito de polizas bajo el codigo pol120190177\n\n', metadata={'num_articles': 14, 'num_tokens': 16037, 'source': 'dataset/raw_chunks/POL120190177.pdf'}),
  0.43430053665049584)]

In [59]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [60]:
# Wrap our vectorstore
llm = ChatOpenAI(temperature=0.3, model_name=MODEL_16K, max_tokens=4_000)

In [61]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [62]:


# retriever_chunks = db.as_retriever(search_type="similarity_score_threshold", 
#                             search_kwargs={"score_threshold": 0.005, "k": 5, "filter":{"source":"filter"},"k":1}, 
#                             allowed_search_types = (
#         "similarity",
#         "similarity_score_threshold",
#         "mmr",
#     ))

retriever_chunks = db.as_retriever(search_type="mmr", 
                            search_kwargs={"k": 10, 
                                           #"filter":{"source":"filter"}
                                           },
)

retriever_summary = db_summarized.as_retriever(search_type="similarity", 
                            search_kwargs={"k": 1},
)

In [63]:
retriever_chunks.get_relevant_documents("Que cubre la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS")

[Document(page_content='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario.\n\nARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado. Lo anterior, cuando al asegurado le ocurra un accidente durante la vigencia de esta póliza q

In [64]:
db.as_retriever(search_type="mmr", 
                            search_kwargs={"k": 10, 
                                           "filter":{"source":"dataset/raw_chunks/POL120190177.pdf"},
                                           }).get_relevant_documents("Que cubre la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS")

[Document(page_content='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario.\n\nARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado. Lo anterior, cuando al asegurado le ocurra un accidente durante la vigencia de esta póliza q

In [65]:
from langchain.prompts import PromptTemplate
template ="""

Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.
Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.
Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.
        
Con base en los siguientes contextos, necesito que me identifiques el numero de poliza asociado
a la siguiente pregunta. Si no sabes la respuesta, simplemente di que no lo sabes, no trates de 
inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.

Pregunta: {question}
Contexto: {context}

El numero de poliza respeta el siguiente formato de 3 letras y 9 digitos, como por ejemplo POL120190177

Numero de poliza mas relevante:"""
qa1_prompt = PromptTemplate(input_variables=["context", "question"],template=template)

In [66]:
template_2 = """"
Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.
Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.
Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.
Utiliza las siguientes partes del contexto para responder la pregunta al final. 
El contexto está hecho de partes de un documento de políticas que contiene varios artículos 

La pregunta es: {question}.

El contexto es: {context}. 
Respuesta útil:"""
qa2_prompt = PromptTemplate(input_variables=["context", "question"],template=template)

In [67]:
# Run chain
from langchain.chains import RetrievalQA


qa_chain1 = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=retriever_summary, 
                                  return_source_documents=True,
                                  chain_type_kwargs={"prompt": qa1_prompt}
)


In [68]:
question = "PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO"
result = qa_chain1(question)
result

{'query': 'PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO',
 'result': 'POL120190177',
 'source_documents': [Document(page_content='poliza de accidentes personales / reembolso gastos medicos\n\nincorporada al deposito de polizas bajo el codigo pol120190177\n\n', metadata={'num_articles': 14, 'num_tokens': 16037, 'source': 'dataset/raw_chunks/POL120190177.pdf'})]}

In [69]:
pattern = r"([A-za-z]{3}\d{3,15})"
policy_num = re.findall(pattern, result['result'])[0]
policy_num.upper()

'POL120190177'

In [70]:
openai_template="""Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.
        Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.
        Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.
        
        Utilizaras el siguiente contexto para responder la pregunta del usuario.
        {context}"""
human_template="{question}"
system_message_prompt = SystemMessagePromptTemplate.from_template(openai_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [71]:
question = "Necesito un resumen de la PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO"

In [72]:
docs = db_summarized.similarity_search(question, k = 1)

In [73]:
docs[0]

Document(page_content='poliza de accidentes personales / reembolso gastos medicos\n\nincorporada al deposito de polizas bajo el codigo pol120190177\n\n', metadata={'num_articles': 14, 'num_tokens': 16037, 'source': 'dataset/raw_chunks/POL120190177.pdf'})

In [74]:
prompt_test= chat_prompt.format_prompt(question=question, context="\n".join([doc.page_content for doc in docs])).to_messages()
prompt_test

[SystemMessage(content="Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.\n        Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.\n        Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.\n        \n        Utilizaras el siguiente contexto para responder la pregunta del usuario.\n        poliza de accidentes personales / reembolso gastos medicos\n\nincorporada al deposito de polizas bajo el codigo pol120190177\n\n", additional_kwargs={}),
 HumanMessage(content='Necesito un resumen de la PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO', additional_kwargs={}, example=False)]

In [97]:
# llm(prompt_test)

AIMessage(content='La póliza de Accidentes Personales / Reembolso Gastos Médicos es un tipo de seguro ofrecido por QuePlan que brinda protección en caso de sufrir un accidente que resulte en lesiones o gastos médicos. Esta póliza tiene como objetivo cubrir los gastos médicos necesarios para la atención de las lesiones sufridas en el accidente.\n\nAl contratar esta póliza, estarás protegido en caso de sufrir accidentes como caídas, fracturas, quemaduras, entre otros. En caso de que ocurra un accidente cubierto por la póliza, podrás recibir un reembolso de los gastos médicos incurridos, siempre y cuando estén dentro de los límites y condiciones establecidos en la póliza.\n\nEs importante tener en cuenta que esta póliza no cubre enfermedades o condiciones médicas preexistentes, ni tampoco accidentes ocurridos como resultado de actividades consideradas peligrosas o imprudentes.\n\nSi estás interesado en contratar esta póliza, te recomiendo que te pongas en contacto con nuestro equipo de at

In [75]:
def custom_filter_chain(question:str) -> str:
    docs = db_summarized.similarity_search(question, k = 1)
    most_likely_doc = docs[0].metadata['source']
    return most_likely_doc

In [76]:
custom_filter_chain(question)

'dataset/raw_chunks/POL120190177.pdf'

In [77]:
docs = db.similarity_search(question, k = 10, filter={"source":custom_filter_chain(question)})
docs

[Document(page_content='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario.\n\nARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado. Lo anterior, cuando al asegurado le ocurra un accidente durante la vigencia de esta póliza q

In [78]:
def custom_qa(question:str) -> dict[str,str,str]:
    response_dict = {}
    docs = db.similarity_search(question, k = 10, filter={"source":custom_filter_chain(question)})
    # Stuffing the content in a single window
    chat_prompt_openai= chat_prompt.format_prompt(question=question, context="\n".join([doc.page_content for doc in docs])).to_messages()
    response = llm(chat_prompt_openai)
    response_dict['response'] = response
    response_dict['docs'] = docs
    response_dict['chat_prompt_openai'] = chat_prompt_openai
    return response_dict

In [79]:
chat_prompt_openai= chat_prompt.format_prompt(question=question, context="\n".join([doc.page_content for doc in docs])).to_messages()
chat_prompt_openai

[SystemMessage(content='Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.\n        Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.\n        Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un \'Sí\' o un \'No\', ni por extenso ni abreviado.\n        \n        Utilizaras el siguiente contexto para responder la pregunta del usuario.\n        PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII

In [80]:
def qa_tool(question:str) -> str:
    response_dict = custom_qa(question)
    response = response_dict['response']
    return response

In [116]:
result_test = custom_qa(question)

In [119]:
result_test

([SystemMessage(content='Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.\n        Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.\n        Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un \'Sí\' o un \'No\', ni por extenso ni abreviado.\n        \n        Utilizaras el siguiente contexto para responder la pregunta del usuario.\n        PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\n\nIncorporada al Depósito de Pólizas bajo el código POL120190177\n\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VII

In [128]:
def pretty_print(input_str):
    # Get the length of the input string
    length = len(input_str)

    # Initialize variables to keep track of the start and end index for each line
    start_index = 0
    end_index = 0

    while end_index < length:
        # Set the end index to the start index plus 100 characters
        end_index = start_index + 100
        
        # Check if the end index exceeds the length of the string
        if end_index >= length:
            end_index = length
        
        # Find the last space within the 100 characters
        last_space = input_str.rfind(' ', start_index, end_index)

        # If there is a space, print up to that space
        if last_space != -1 and last_space > start_index:
            print(input_str[start_index:last_space])
            start_index = last_space + 1
        else:
            # If there is no space, print the whole 100 characters
            print(input_str[start_index:end_index])
            start_index = end_index

In [126]:
_ , _ , text = result_test


In [129]:
pretty_print(text.content)

La póliza de Accidentes Personales / Reembolso Gastos Médicos de QuePlan ofrece cobertura para los
gastos médicos razonables y acostumbrados que se incurran como resultado de un accidente durante la
vigencia de la póliza. La compañía aseguradora reembolsará al asegurado o pagará directamente al
prestador de salud los gastos médicos una vez se haya otorgado y pagado la cobertura del sistema de
salud previsional, seguros complementarios u otros beneficios contratados por el asegurado.

Para
que los gastos sean reembolsados, deben cumplirse los siguientes requisitos: el accidente debe
ocurrir durante la vigencia de la póliza, los gastos deben realizarse dentro del período de
duración de reembolso, el monto de los gastos debe superar el deducible establecido en las
condiciones particulares de la póliza, la suma de los reembolsos de gastos realizados durante la
vigencia anual de la póliza no debe superar el monto máximo de gastos reembolsables establecido,
los gastos no deben provenir de un

In [81]:
from langchain.tools import DuckDuckGoSearchRun , DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()
def duck_duck_wrapper(query):
    search_results = search.run(f"{query}")
    return search_results 

In [82]:
duck_duck_wrapper("colombia")

"Colombia, second-greatest biodiversity in the world Allow us to introduce you to the unique country of Colombia. A haven of biodiversity, with more than 50,000 species of animals and plants. Beautiful nature, exotic animals and a variety of orchid flowers will convince you that Colombia is one of the most extraordinary places on Earth. People demonstrate for the legalization of cannabis in Bogota, Colombia on May 6, 2023. On a recent Friday morning, about 200 coca and marijuana farmers gathered in the small town of Cajibio ... Colombia will meet South Korea in the 2023 Women's World Cup on Monday night from the Sydney Football Stadium. Both teams are eager to make their mark on the global stage, with Colombia makin… Colombia is the most populous nation of Spanish-speaking South America. More than one-third of its inhabitants live in the six largest metropolitan areas, of which Bogotá is the largest. The nation's political instability has been historically tied to the unequal distribut

In [83]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.chains import RetrievalQA

In [136]:
question = "Que cubre el seguro PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO"

In [89]:


tools = [
    Tool(
        name = "Web Search",
        func=duck_duck_wrapper,
        description="useful for when you need to answer questions about current events, news or the current state of the world. Do not use it when asked about policies, in that case, use the policy search tool",
    ),
    Tool(
        name = "Policy Search",
        func=qa_tool,
        description="useful for when you need to answer questions about an specific policy NUMBER, in that case, use the policy search tool, if you dont have the policy number, ask the user for more information",
    ),
]


In [90]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [195]:
from langchain.chains import LLMChain

llm_template = """Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.
        Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.
        Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.

        
        Utilzaras la siguiente informacion tipo REACT para responder la pregunta del usuario.
        {agent_scratchpad}
"""
llm_prompt = PromptTemplate(input_variables=["agent_scratchpad"], template=template)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. 'question' (type=value_error)

In [84]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [92]:
# Set up the base template
template = """Eres un agente comercial de polizas de la empresa QuePlan. Eres encargado de responder las preguntas de los clientes sobre las polizas de la empresa.
Si no sabes la respuesta, simplemente di que no lo sabes, no trates de inventar una respuesta. Si te falta información, pide al usuario que la proporcione en la siguiente pregunta.
Crea una respuesta con la suficiente información para que el usuario pueda entender la respuesta. No te limites a responder con un 'Sí' o un 'No', ni por extenso ni abreviado.
        
Como agente tienes acceso a las siguientes herramientas:

{tools}

Utiliza unicamente el siguiente formato para tus respuestas:

Question: La pregunta que te hace el usuario
Thought: Piensa y razona como proceder para resolver la pregunta
Action: La mejor accion a realizar. Debe ser unicamente una de las siguientes [{tool_names}]
Action Input: Con base en la accion, que input necesita la accion para ejecutarse
Observation: El resultado de la accion
... (Este ciclo de Thought/Action/Action Input/Observation se puede repetir N veces hasta encontrar la respuesta final) 
Thought: Creo que se la respuesta final
Final Answer: La respuesta final que le daras al usuario

Empieza a responder la pregunta del usuario: Responde siempre de forma profesional y educada.

Question: {input}
{agent_scratchpad}"""

In [93]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [94]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [98]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [100]:
output_parser = CustomOutputParser()

In [97]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [99]:
tool_names = [tool.name for tool in tools]
tool_names

['Web Search', 'Policy Search']

In [101]:
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [102]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)



In [103]:
agent_executor.run(question)





> Entering new AgentExecutor chain...
Thought: Para responder a esta pregunta, necesito buscar información sobre la póliza de accidentes personales / reembolso gastos médicos de la empresa QuePlan. Utilizaré la herramienta de búsqueda de políticas para obtener los detalles específicos de esta póliza.

Action: Policy Search
Action Input: PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO

Observation:content='La póliza de Accidentes Personales / Reembolso Gastos Médicos de QuePlan es un contrato de seguro que cubre los gastos médicos razonables y acostumbrados en caso de accidente. La compañía aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los gastos médicos incurridos una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado.\n\nEsta cobertura aplica cuando el asegurado sufre un accidente durante la vigencia de la póliza que requiera su internación h

'La póliza de Accidentes Personales / Reembolso Gastos Médicos de QuePlan es un contrato de seguro que cubre los gastos médicos razonables y acostumbrados en caso de accidente. La compañía aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los gastos médicos incurridos una vez se haya otorgado y pagado la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado.\n\nEsta cobertura aplica cuando el asegurado sufre un accidente durante la vigencia de la póliza que requiera su internación hospitalaria y cuando estos gastos se realicen en los prestadores de salud determinados por el asegurador. Para que los gastos sean reembolsados, se deben cumplir ciertos requisitos, como que el accidente haya ocurrido durante la vigencia de la póliza, los gastos se hayan incurrido dentro del período de duración de reembolso, el monto de los gastos supere el deducible establecido, la suma de los reembolsos de gastos n

In [104]:
question = "Cual es la diferencia entre la poliza de accidentes personales y la poliza de accidentes laborales"

'Necesito un resumen de la PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICO'